In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel
from tqdm import tqdm
import os
import time
from transformers import BertTokenizer
from transformers import logging
import processing
from sklearn import metrics
import MyBERT

In [ ]:

# 训练准备阶段，设置超参数和全局变量
file_name = 'readme.md'
batch_size = 16
num_epoch = 10  # 训练轮次
check_step = 1  # 用以训练中途对模型进行检验：每check_step个epoch进行一次测试和保存模型

learning_rate = 1e-5  # 优化器的学习率

# 获取训练、测试数据、分类类别总数
train_data = processing.get_exist2021_data_temp(type='train', len=3437)
test_data = processing.get_exist2021_data_temp(type='test', len=1000)
categories = 2

train_iter, test_iter = MyBERT.load_bert_data(train_data, test_data, batch_size)

#固定写法，可以牢记，cuda代表Gpu
# torch.cuda.is_available()可以查看当前Gpu是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练模型，因为这里是英文数据集，需要用在英文上的预训练模型：bert-base-uncased
# uncased指该预训练模型对应的词表不区分字母的大小写
# 详情可了解：https://huggingface.co/bert-base-uncased
pretrained_model_name = 'bert-base-uncased'
# 创建模型 BertSST2Model
model = MyBERT.MyBertModel(categories, pretrained_model_name)
# 固定写法，将模型加载到device上，
# 如果是GPU上运行，此时可以观察到GPU的显存增加
model.to(device)

# 训练过程
# Adam是最近较为常用的优化器，详情可查看：https://www.jianshu.com/p/aebcaf8af76e
optimizer = Adam(model.parameters(), learning_rate)  # 使用Adam优化器
loss = nn.CrossEntropyLoss()  # 使用crossentropy作为二分类任务的损失函数

# 记录当前训练时间，用以记录日志和存储
timestamp = time.strftime("%m_%d_%H_%M", time.localtime())


In [ ]:
fp = open(file_name, 'a+')
for epoch in range(1, num_epoch + 1):
    MyBERT.train(model, train_iter,device, optimizer, loss, epoch)
    MyBERT.test(model, test_iter, device, epoch, file_name)